* Initialize the libraries and variables
* Add `AEM_TOKEN` to the `.env` file

In [ ]:
import { logError } from "./logError.js";
import { reloadEnv } from "./reloadEnv.js"
await reloadEnv();

const token = Deno.env.get("AEM_TOKEN");
const aemOrigin = 'https://author-p22655-e59433.adobeaemcloud.com';
const fileListFilename = 'cc_filelist.txt';
const format = 'html';

* Test if the token is working

In [ ]:
const url = `${aemOrigin}/bin/querybuilder.json?path=/content/dam/mas&path.flat=true&type=sling:Folder&p.limit=-1`;

const response = await fetch(url, {
  headers: {
    "Authorization": `Bearer ${token}`
  }
});

if (!response.ok) {
  logError(response);
}

console.log(response.status);

* Read the list of file paths

In [ ]:
const fileList = (await Deno.readTextFile(fileListFilename)).split('\n');
console.log(`File count: ${fileList.length}`);

* Parse the HTMLs or MDs into fields 

In [ ]:
import { DOMParser } from "https://deno.land/x/deno_dom@v0.1.38/deno-dom-wasm.ts";

const getText = (element) => element?.textContent?.trim() || '';
const getHref = (element) => element?.getAttribute('href') || '';

const extractOsi = (url) => {
  try {
    const urlObj = new URL(url);
    return urlObj.searchParams.get('osi') || '';
  } catch (e) {
    return '';
  }
};

const ext = {'md': '.md', 'html': '.plain.html'}[format];
const fileFolder = {'md': './md', 'html': './html'}[format];

const items = [];
const itemErrors = [];

function processCta(ctaLinks, filePath) {
  const ctas = [];
  let osi = '';

  if (ctaLinks.length > 3 || ctaLinks.length === 0) {
    throw new Error(`${filePath}: Got ${ctaLinks.length} CTAs\n${ctaChild.outerHTML}`);
  }

  for (const ctaLink of ctaLinks) {
    const href = getHref(ctaLink);
    const [text, aria] = getText(ctaLink).split('|').map(s => s.trim());
    ctas.push({
      text,
      aria,
      href: href,
      osi: extractOsi(href),
      parentTag: ctaLink.parentElement.tagName,
    });
  }

  const osiCtas = ctas.filter(cta => cta.osi);
  if (osiCtas.length > 0) {
    osi = osiCtas[0].osi;
  }

  return { ctas, osi };
}

for (let i = 0; i < fileList.length; i++) {
  const filePath = fileList[i];
  const localPath = `${fileFolder}${filePath}${ext}`;

  const htmlContent = await Deno.readTextFile(localPath);

  const doc = new DOMParser().parseFromString(htmlContent, 'text/html');

  const rows = doc.querySelectorAll('.merch-card.catalog > div > div');

  if (rows.length === 0) {
    itemErrors.push(filePath);
    continue;
  }

  const result = {
    filePath,
    column1Html: '',
    deviceTypes: [],
    recommendedFor: [],
    mnemonicIcons: [],
    cardTitle: '',
    cardTitleLink: '',
    description: '',
    ctas: [],
    ctasHtml: '',
    osi: '',
    tags: [],
  };

  let rowIndex = 0;

  if (rows.length > 2) {
    // Row 1: Device types and Recommended for
    const row1 = rows[rowIndex++];
    // Clean up excess whitespaces
    result.column1Html = row1.innerHTML
      .split('\n')
      .map(line => line.trim())
      .filter(line => line.length > 0)
      .join('');
    const lists = row1.querySelectorAll('ul');
    if (lists.length >= 1) {
      const deviceItems = lists[0].querySelectorAll('li');
      result.deviceTypes = Array.from(deviceItems).map(li => getText(li));
    }
    if (lists.length >= 2) {
      const recItems = lists[1].querySelectorAll('li');
      result.recommendedFor = Array.from(recItems).map(li => getText(li));
    }
  }

  if (rows.length >= 2) {
    // Row 2: Main content with 4 or 5 children
    const row2 = rows[rowIndex++];
    const children = Array.from(row2.children);

    // Child 1: Mnemonic icons (could be multiple icons, each with its link)
    if (children.length >= 1) {
      const iconLinks = children[0].querySelectorAll('a');
      iconLinks.forEach(iconLink => {
        const text = getText(iconLink);
        const parts = text.split('|').map(s => s.trim());
        result.mnemonicIcons.push({
          icon: parts[0] || '',
          alt: parts[1] || '',
          link: getHref(iconLink)
        });
      });
    }

    // Child 2: Title and title link (h3)
    if (children.length >= 2) {
      const titleLink = children[1].querySelector('a');
      if (titleLink) {
        result.cardTitle = getText(titleLink);
        result.cardTitleLink = getHref(titleLink);
      } else {
        result.cardTitle = getText(children[1]);
      }
    }

    // Child 3: Description (includes description text and learn more link)
    if (children.length >= 3) {
      const descChild = children[2];
      result.description = descChild.outerHTML;
    }

    // Child 4: Footer CTAs (strong and em links)
    if (children.length >= 4) {
      const ctaChild = children[3];
      result.ctasHtml = ctaChild.innerHTML;
      
      const ctaLinks = ctaChild.querySelectorAll('a');
      const { ctas, osi } = processCta(ctaLinks, filePath);
      result.ctas = ctas;
      result.osi = osi;
    }

    if (children.length >= 5) {
      // Move line 4 to description
      const learnmore = result.ctas.map(x => `<a class="primary-link" href="${x.href}" aria-label="${x.aria}">${x.text}</a>`).join(' | ');
      result.description = result.description.replace(new RegExp('</p>$'), `<br>${learnmore}</p>`);

      // Process the new CTAs
      const ctaChild = children[4];
      result.ctasHtml = ctaChild.innerHTML;

      const ctaLinks = ctaChild.querySelectorAll('a');
      const { ctas, osi } = processCta(ctaLinks, filePath);
      result.ctas = ctas;
      result.osi = osi;      
    }
  }

  if (rows.length >= 2) {
    // Row 3: Tags/categories
    const row3 = rows[rowIndex++];
    const tagParagraphs = row3.querySelectorAll('p');
    result.tags = Array.from(tagParagraphs).map(p => getText(p));
  }

  items.push(result);
}

console.log(`Read ${items.length} cards`);
console.log(`Errors: ${itemErrors.length}\n${itemErrors.join('\n')}`);

* Review the parsed items

In [ ]:
console.log(JSON.stringify(items[0], null, 2));

* Normalize the items for writing to ODIN
* Some fields need be proccessed first

In [ ]:
const normItems = [];
for (let i = 0; i < items.length; i++) {
  // Clone the item
  const item = JSON.parse(JSON.stringify(items[i]));

  // Update the mnemonic icons links to hlx.page/aem.page to aem.live

  item.mnemonicIcons.forEach(x => {
    x.icon = x.icon.replace('hlx.page', 'aem.live').replace('aem.page', 'aem.live');
  });
  
  // Update the card description
  const descDoc = new DOMParser().parseFromString(item.description, 'text/html');
  const descLinks = descDoc.querySelectorAll('a');
  if (descLinks.length > 0) {
    descLinks.forEach(x => {
      const [text, aria] = x.textContent.split('|').map(s => s.trim());
      x.textContent = text;
      x.setAttribute('aria-label', aria);
    });
  }
  item.description = descDoc.body.innerHTML;

  // Update the ctas text
  item.ctas.forEach(x => {
    const ctaText = x.text.split('|').map(s => s.trim());
    x.text = ctaText[0];
    x.alt = ctaText[1];
  });

  item.ctasHtml = item.ctas.map(x => {
    const attrs = [];
    let className = { 'strong': 'accent', 'em': 'primary-outline' }[x.parentTag.toLowerCase()];
    if (className) {
      attrs.push(`class="${className}"`);
    } else {
      attrs.push(`class="primary-link"`);
    }
    if (x.href) {
      attrs.push(`href="${x.href}"`);
    }
    if (x.aria) {
      attrs.push(`aria-label="${x.aria}"`);
    }
    return `<a ${attrs.join(' ')}>${x.text}</a>`;
  }).join(' ');
  
  normItems.push(item);
}



* Review the normalized items

In [ ]:
console.log(JSON.stringify(normItems[0], null, 2));

* Write to ODIN
* Update `parentPath` if needed

In [ ]:
const url = `${aemOrigin}/adobe/sites/cf/fragments`;
const parentPath = `/content/dam/mas/sandbox/en_US`;

const odinResults = [];
const odinErrors = [];

//for (let i=0; i<results.length; i++) {
for (let i = 28; i < 29; i++) {
  const result = normItems[i];

  const title = result.filePath;
  const description = `imported from Milo`;
  const name = `imported-from-milo-${i}`;
  const modelId = 'L2NvbmYvbWFzL3NldHRpbmdzL2RhbS9jZm0vbW9kZWxzL2NhcmQ';

  const fields = [
    {
      "name": "variant",
      "type": "text",
      "multiple": false,
      "locked": false,
      "values": [
        "catalog"
      ]
    },
    {
      "name": "size",
      "type": "text",
      "multiple": false,
      "locked": false,
      "values": [
        "Default"
      ]
    },
    {
      "name": "cardTitle",
      "type": "text",
      "multiple": false,
      "locked": false,
      "values": [ result.cardTitle ]
    },
    {
      "name": "mnemonicIcon",
      "type": "text",
      "multiple": true,
      "locked": false,
      "values":  result.mnemonicIcons.map(icon => icon.icon)

    },
    {
      "name": "mnemonicLink",
      "type": "text",
      "multiple": true,
      "locked": false,
      "values": result.mnemonicIcons.map(icon => icon.link)
    },
    {
      "name": "shortDescription",
      "type": "long-text",
      "multiple": false,
      "locked": false,
      "mimeType": "text/html",
      "values": [
        result.shortDescription
      ]
    },
    {
      "name": "description",
      "type": "long-text",
      "multiple": false,
      "locked": false,
      "mimeType": "text/html",
      "values": [
        result.description
      ]
    },
    {
      "name": "ctas",
      "type": "long-text",
      "multiple": false,
      "locked": false,
      "mimeType": "text/html",
      "values": [
        result.ctasHtml
      ]
    },
    {
      "name": "osi",
      "type": "text",
      "multiple": false,
      "locked": false,
      "values": [
        result.osi
      ]
    },
  ];

  const response = await fetch(url, {
    method: 'POST',
    headers: {
      'Content-Type': 'application/json',
      "x-api-key": "mas-studio",
      "Authorization": `Bearer ${token}`,
      'x-aem-affinity-type': 'api',
    },
    body: JSON.stringify({
      title,
      name,
      modelId,
      parentPath,
      description,
      fields,
    }),
  })

  if (!response.ok) {
    odinErrors.push(response);
  } else {
    const data = await response.json();
    odinResults.push(data);
  }
}

console.log(`Imported ${odinResults.length} cards`);
console.log(`Errors: ${odinErrors.length}`);


* List IDs of imported cards 

In [ ]:
const ids = odinResults.map(x => x.id);
console.log(ids);

* Check the imported card
* Update `cardId`

In [ ]:
const cardId = ids[0];
const url = `${aemOrigin}/adobe/sites/cf/fragments/${cardId}?references=direct-hydrated`;

const response = await fetch(url, {
  headers: {
    "x-api-key": "mas-studio",
    "Authorization": `Bearer ${token}`,
    pragma: 'no-cache',
    'cache-control': 'no-cache',
    'x-aem-affinity-type': 'api',
  }
});

if (!response.ok) {
  logError(response);
}

const data = await response.json();
await Deno.writeTextFile('catalog.json', JSON.stringify(data, null, 2));
console.log(JSON.stringify(data, null, 2));